In [13]:
# imports 
import pandas as pd 
import numpy as np
from data_extraction import DataExtractionHelper

# load helper 
helper = DataExtractionHelper() 
# remove max rows for display output 
pd.set_option('display.max_columns', None)

In [14]:
# read in clinvar data 
variant_df = helper.read_in_data('../data/variant_summary.txt', 'txt', '\t')
variant_df

c:\Users\prodb\Documents\Projects\hive\BiomarkerKB\src\data_extraction.py:99: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(input_file, sep = delim)


,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline;unknown,germline,GRCh37,NC_000007.13,7,4820844,4820847,na,na,7p22.1,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4820844,GGAT,TGCTGTAAACTGTAACTGTAAA
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,-,RCV000000012,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline;unknown,germline,GRCh38,NC_000007.14,7,4781213,4781216,na,na,7p22.1,"criteria provided, single submitter",2,-,N,"ClinGen:CA215070,OMIM:613653.0001",3,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,-,RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline,germline,GRCh37,NC_000007.13,7,4827361,4827374,na,na,7p22.1,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4827360,GCTGCTGGACCTGCC,G
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,-,RCV000000013,"MONDO:MONDO:0013342,MedGen:C3150901,OMIM:61364...",Hereditary spastic paraplegia 48,germline,germline,GRCh38,NC_000007.14,7,4787730,4787743,na,na,7p22.1,no assertion criteria provided,1,-,N,"ClinGen:CA215072,OMIM:613653.0002",1,3,4787729,GCTGCTGGACCTGCC,G
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,-,RCV000000014,"MONDO:MONDO:0033005,MedGen:C4551772,OMIM:25130...",Galloway-Mowat syndrome 1,germline,germline,GRCh37,NC_000015.9,15,85342440,85342440,na,na,15q25.3,no assertion criteria provided,1,-,N,"OMIM:613624.0001,ClinGen:CA210674,UniProtKB:Q9...",1,4,85342440,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324858,2403615,Duplication,NM_013275.6(ANKRD11):c.2022dup (p.Glu675fs),29123,ANKRD11,HGNC:21316,Pathogenic,1,"Oct 02, 2019",-1,-,RCV003128077,MedGen:C0008073,Developmental disorder,de novo,germline,GRCh38,NC_000016.10,16,89284519,89284520,na,na,16q24.3,"criteria provided, single submitter",1,-,N,-,2,2430112,89284519,C,CT
4324859,2403616,single nucleotide variant,NM_032892.5(FRMD5):c.285G>C (p.Lys95Asn),84978,FRMD5,HGNC:28214,Uncertain significance,0,"Feb 16, 2023",-1,-,RCV003128078,"MedGen:CN322365,OMIM:620094",NEURODEVELOPMENTAL DISORDER WITH EYE MOVEMENT ...,de novo,germline,GRCh37,NC_000015.9,15,44211701,44211701,na,na,15q15.3,"criteria provided, single submitter",1,-,N,-,2,2430113,44211701,C,G
4324860,2403616,single nucleotide variant,NM_032892.5(FRMD5):c.285G>C (p.Lys95Asn),84978,FRMD5,HGNC:28214,Uncertain significance,0,"Feb 16, 2023",-1,-,RCV003128078,"MedGen:CN322365,OMIM:620094",NEURODEVELOPMENTAL DISORDER WITH EYE MOVEMENT ...,de novo,germline,GRCh38,NC_000015.10,15,43919503,43919503,na,na,15q15.3,"criteria provided, single submitter",1,-,N,-,2,2430113,43919503,C,G
4324861,2403617,single nucleotide variant,NM_001165963.4(SCN1A):c.5158A>G (p.Ile1720Val),6323,SCN1A,HGNC:10585,Uncertain significance,0,-,-1,-,RCV003128079,"MONDO:MONDO:0100135,MedGen:C0751122,OMIM:60720...",Severe myoclonic epilepsy in infancy,germline,germline,GRCh37,NC_000002.11,2,166848627,166848627,na,na,-,

In [15]:
# drop GRCh37 rows 
variant_df.drop(variant_df[variant_df['Assembly'] == 'GRCh37'].index, inplace = True)
variant_df.shape

(2136789, 34)

In [16]:
# unravel out phenotype list column 
variant_df = helper.split_col_on_delim(df = variant_df, split_column = 'PhenotypeList', new_col_name = 'Disease', delim = '|')
variant_df.shape

(2838403, 34)

In [17]:
# 2nd level unravel 
variant_df = helper.split_col_on_delim(df = variant_df, split_column = 'Disease', new_col_name = 'disease', delim = ';')
variant_df.shape

(3101632, 34)

In [18]:
# only keep rows pertaining to cancer or carcinoma data 
variant_df = variant_df[variant_df['disease'].str.contains('cancer|carcinoma', regex = True, case = False)]
variant_df.shape

(220938, 34)

In [19]:
# replace all -1 values with None 
variant_df.loc[variant_df['RS# (dbSNP)'] == -1, 'RS# (dbSNP)'] = None
variant_df['RS# (dbSNP)'].value_counts(dropna = False)

NaN             62935
5.877787e+08       46
1.154065e+07       44
2.893458e+07       42
7.862011e+08       41
                ...  
1.333599e+09        1
5.592276e+08        1
8.998257e+08        1
9.916833e+08        1
1.748147e+09        1
Name: RS# (dbSNP), Length: 96397, dtype: int64

In [20]:
# reset the dataframe index
variant_df.reset_index(drop = True, inplace = True)
variant_df

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,disease
0,15048,single nucleotide variant,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),3077,HFE,HGNC:4886,Conflicting interpretations of pathogenicity; ...,1,"Nov 03, 2022",1800562.0,-,RCV000000019|RCV000000020|RCV000000022|RCV0000...,"MONDO:MONDO:0021001,MedGen:C3469186,OMIM:23520...",germline;unknown,germline,GRCh38,NC_000006.12,6,26092913,26092913,na,na,6p22.2,"criteria provided, conflicting interpretations",37,"ACMG2021,ACMG2022",Y,"UniProtKB:Q30201#VAR_004398,OMIM:613609.0001,C...",3,9,26092913,G,A,Hereditary cancer-predisposing syndrome
1,15147,single nucleotide variant,NM_017849.4(TMEM127):c.475C>T (p.Gln159Ter),55654,TMEM127,HGNC:26038,Pathogenic,1,"Aug 27, 2021",121908830.0,-,RCV000000128|RCV000114833|RCV000164900|RCV0013...,MedGen:C3149711|Human Phenotype Ontology:HP:00...,germline,germline,GRCh38,NC_000002.12,2,96254050,96254050,na,na,2q11.2,"criteria provided, multiple submitters, no con...",4,"ACMG2021,ACMG2022",N,"ClinGen:CA113855,OMIM:613403.0002",3,108,96254050,G,A,Hereditary cancer-predisposing syndrome
2,15375,single nucleotide variant,NM_001042492.3(NF1):c.4330A>G (p.Lys1444Glu),4763,NF1,HGNC:7765,Pathogenic,1,"Sep 27, 2022",137854550.0,-,RCV000000364|RCV000489593|RCV000626644|RCV0007...,"MONDO:MONDO:0018975,MedGen:C0027831,OMIM:16220...",germline;inherited;somatic;unknown,germline/somatic,GRCh38,NC_000017.11,17,31258500,31258500,na,na,17q11.2,"criteria provided, multiple submitters, no con...",16,-,N,"ClinGen:CA251440,OMIM:613113.0005",3,336,31258500,A,G,Hereditary cancer-predisposing syndrome
3,15382,single nucleotide variant,NM_001042492.3(NF1):c.5902C>T (p.Arg1968Ter),4763,NF1,HGNC:7765,Pathogenic,1,"Sep 10, 2022",137854552.0,-,RCV000000371|RCV000418287|RCV000492774|RCV0007...,"MONDO:MONDO:0018975,MedGen:C0027831,OMIM:16220...",de novo;germline;inherited;unknown,germline,GRCh38,NC_000017.11,17,31334927,31334927,na,na,17q11.2,"criteria provided, multiple submitters, no con...",14,-,N,"ClinGen:CA325489,OMIM:613113.0012",3,343,31334927,C,T,Hereditary cancer-predisposing syndrome
4,15383,single nucleotide variant,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),4763,NF1,HGNC:7765,Pathogenic,1,"Oct 31, 2022",137854560.0,-,RCV000000372|RCV000483061|RCV000492495|RCV0025...,"MONDO:MONDO:0018975,MedGen:C0027831,OMIM:16220...",germline;somatic;unknown,germline/somatic,GRCh38,NC_000017.11,17,31249093,31249093,na,na,17q11.2,"criteria provided, multiple submitters, no con...",15,-,N,"ClinGen:CA325494,OMIM:613113.0027",3,344,31249093,C,T,Hereditary cancer-predisposing syndrome
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220933,2192418,single nucleotide variant,NM_005732.4(RAD50):c.3668T>G (p.Leu1223Arg),10111,RAD50,HGNC:9816,Uncertain significance,0,"May 31, 2022",NaN,-,RCV003066002,"MONDO:MONDO:0015356,MeSH:D009386,MedGen:C00276...",germline,germline,GRCh38,NC_000005.10,5,132640721,132640721,na,na,5q31.1,"criteria provided, single submitter",1,-,N,-,2,2134861,132640721,T,G,Hereditary cancer-predisposing syndrome
220934,2401322,Insertion,NM_000059.4(BRCA2):c.6916_6917insA (p.Ala2306fs),675,BRCA2,HGNC:1101,Pathogenic,1,-,NaN,-,RCV002789934,"MONDO:MONDO:0016419,MedGen:C0346153,OMIM:11448...",germline,germline,GRCh38,NC_000013.11,13,32344632,32344633,na,na,13q13.1,no assertion criteria provided,1,"ACMG2013,ACMG2016,ACMG2021,ACMG2022",N,-,2,2412590,32344632,G,GA,Familial cancer of breast
220935,2402684,single nucleotide variant,NM_000059.4(BRCA2):c.7171G>T (p.Glu2391Ter),675,BRCA2,HGNC:1101,Likely pathogenic,1,"Jan 06, 2023",NaN,-,RCV0

In [21]:
# rs_id column 
helper.populate_col(source = variant_df['RS# (dbSNP)'], target_col = 'rs_id', dtype = 'Int64')
# mutation column
helper.populate_col(source = variant_df['Name'], target_col = 'mutation')
# name column 
helper.populate_col(source = variant_df['VariationID'], target_col = 'name')
# gene column 
helper.populate_col(source = variant_df['GeneSymbol'], target_col = 'gene')
# disease column 
helper.populate_col(source = variant_df['disease'], target_col = 'disease')
# variation column 
helper.populate_col(source = variant_df['VariationID'], target_col = 'variation')
# biomarker status column 
bio_status_series = pd.Series('presence of', index = helper.biomarker_df.index)
helper.populate_col(source = bio_status_series, target_col = 'biomarker_status')
# best biomarker type column
biomarker_type_series = pd.Series('risk_biomarker', index = helper.biomarker_df.index)
helper.populate_col(source = biomarker_type_series, target_col = 'best_biomarker_type')
# assessed entity type column
assessed_entity_series = pd.Series('gene', index = helper.biomarker_df.index)
helper.populate_col(source = assessed_entity_series, target_col = 'assessed_entity_type')

helper.biomarker_df

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id,gene,disease,uniprot,name,system,doid,mutation,variation
0,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,1800562,HFE,Hereditary cancer-predisposing syndrome,NaN,9,NaN,NaN,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),9
1,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,121908830,TMEM127,Hereditary cancer-predisposing syndrome,NaN,108,NaN,NaN,NM_017849.4(TMEM127):c.475C>T (p.Gln159Ter),108
2,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,137854550,NF1,Hereditary cancer-predisposing syndrome,NaN,336,NaN,NaN,NM_001042492.3(NF1):c.4330A>G (p.Lys1444Glu),336
3,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,137854552,NF1,Hereditary cancer-predisposing syndrome,NaN,343,NaN,NaN,NM_001042492.3(NF1):c.5902C>T (p.Arg1968Ter),343
4,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,137854560,NF1,Hereditary cancer-predisposing syndrome,NaN,344,NaN,NaN,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220933,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,RAD50,Hereditary cancer-predisposing syndrome,NaN,2134861,NaN,NaN,NM_005732.4(RAD50):c.3668T>G (p.Leu1223Arg),2134861
220934,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,BRCA2,Familial cancer of breast,NaN,2412590,NaN,NaN,NM_000059.4(BRCA2):c.6916_6917insA (p.Ala2306fs),2412590
220935,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,BRCA2,Hereditary breast ovarian cancer syndrome,NaN,2429251,NaN,NaN,NM_000059.4(BRCA2):c.7171G>T (p.Glu2391Ter),2429251
220936,NaN,NaN,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,BRCA2,Hereditary breast ovarian cancer syndrome,NaN,2429268,NaN,NaN,NM_000059.4(BRCA2):c.3189del (p.Gln1063fs),2429268


In [22]:
# format the main_x_ref based off of the rs_id column 
rsid_nan_mask = helper.biomarker_df['rs_id'].isna()
x_ref_series = 'dbSNP:rs' + helper.biomarker_df['rs_id'].astype('str')
x_ref_series[rsid_nan_mask] = 'dbSNP:rs<NA>'
helper.populate_col(source = x_ref_series, target_col = 'main_x_ref')
helper.biomarker_df

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id,gene,disease,uniprot,name,system,doid,mutation,variation
0,NaN,dbSNP:rs1800562,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,1800562,HFE,Hereditary cancer-predisposing syndrome,NaN,9,NaN,NaN,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),9
1,NaN,dbSNP:rs121908830,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,121908830,TMEM127,Hereditary cancer-predisposing syndrome,NaN,108,NaN,NaN,NM_017849.4(TMEM127):c.475C>T (p.Gln159Ter),108
2,NaN,dbSNP:rs137854550,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,137854550,NF1,Hereditary cancer-predisposing syndrome,NaN,336,NaN,NaN,NM_001042492.3(NF1):c.4330A>G (p.Lys1444Glu),336
3,NaN,dbSNP:rs137854552,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,137854552,NF1,Hereditary cancer-predisposing syndrome,NaN,343,NaN,NaN,NM_001042492.3(NF1):c.5902C>T (p.Arg1968Ter),343
4,NaN,dbSNP:rs137854560,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,137854560,NF1,Hereditary cancer-predisposing syndrome,NaN,344,NaN,NaN,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220933,NaN,dbSNP:rs<NA>,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,RAD50,Hereditary cancer-predisposing syndrome,NaN,2134861,NaN,NaN,NM_005732.4(RAD50):c.3668T>G (p.Leu1223Arg),2134861
220934,NaN,dbSNP:rs<NA>,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,BRCA2,Familial cancer of breast,NaN,2412590,NaN,NaN,NM_000059.4(BRCA2):c.6916_6917insA (p.Ala2306fs),2412590
220935,NaN,dbSNP:rs<NA>,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,BRCA2,Hereditary breast ovarian cancer syndrome,NaN,2429251,NaN,NaN,NM_000059.4(BRCA2):c.7171G>T (p.Glu2391Ter),2429251
220936,NaN,dbSNP:rs<NA>,NaN,presence of,risk_biomarker,NaN,NaN,NaN,gene,NaN,NaN,<NA>,BRCA2,Hereditary breast ovarian cancer syndrome,NaN,2429268,NaN,NaN,NM_000059.4(BRCA2):c.3189del (p.Gln1063fs),2429268


In [27]:
# uniprot column 
gene_to_uniprot_dict = dict(zip(helper.uniprot_df['Gene Names'].str.lower().str.strip(), helper.uniprot_df['Entry']))
gene_to_uniprot_series = helper.biomarker_df['gene'].str.lower().str.strip().map(gene_to_uniprot_dict).fillna('Null')
helper.populate_col(source = gene_to_uniprot_series, target_col = 'uniprot')

# name column 
name_to_uniprot_dict = dict(zip(helper.uniprot_df['Gene Names'].str.lower().str.strip(), helper.uniprot_df['Protein names']))
name_to_uniprot_series = helper.biomarker_df['gene'].str.lower().str.strip().map(name_to_uniprot_dict)
helper.populate_col(source = name_to_uniprot_series, target_col = 'name')

# loinc column 
loinc_dict = helper.loinc_df.set_index('first_word')['LOINC_NUM'].to_dict()
loinc_series = helper.biomarker_df['gene'].map(loinc_dict)
helper.populate_col(source = loinc_series, target_col = 'loinc_code')

# system column 
system_dict = helper.loinc_df.set_index('LOINC_NUM')['SYSTEM'].to_dict()
system_series = helper.biomarker_df['loinc_code'].map(system_dict)
helper.populate_col(source = system_series, target_col = 'system')

# specimen type column 
uberon_dict = helper.uberon_map.set_index('system').apply(lambda row: f"{row['name']} (UN:{row['uberon_value']})", axis = 1).to_dict()
uberon_series = helper.biomarker_df['system'].map(uberon_dict)
helper.populate_col(source = uberon_series, target_col = 'specimen_type')

# doid column 
doid_dict = helper.doid_map.set_index('Disease Name')['DOID'].to_dict()
doid_series = helper.biomarker_df['disease'].str.lower().map(doid_dict)
# doid synonyms 
doid_synonym_dict = helper.doid_map.set_index('synonym')['DOID'].to_dict()
doid_synonym_series = helper.biomarker_df['disease'].str.lower().map(doid_synonym_dict)
combined_doid_series = doid_series.combine_first(doid_synonym_series)
helper.populate_col(source = combined_doid_series, target_col = 'doid')

# condition column
condition_name_series = helper.biomarker_df['disease'].str.lower() + 'DOID:' + helper.biomarker_df['doid'].astype(str) + ')'
condition_name_series = np.where(helper.biomarker_df['doid'].notna(), condition_name_series, helper.biomarker_df['disease'].str.lower()) 
helper.populate_col(source = condition_name_series, target_col = 'condition_name')

# evidence source column 
evidence_series = pd.Series('ClinVar', index = helper.biomarker_df.index)
evidence_mask = helper.biomarker_df['variation'].notna()
evidence_series[evidence_mask] = 'ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/variation/' + helper.biomarker_df['variation'][evidence_mask].astype(str) + '/?new_evidence=true'
helper.populate_col(source = evidence_series, target_col = 'evidence_source')

# assessed biomarker entity column 
bio_entity_series = 'mutation in ' + helper.biomarker_df['name'] + '(' + helper.biomarker_df['gene'] + ')'
entity_mask = helper.biomarker_df['rs_id'].notna() 
bio_entity_series[entity_mask] = 'rs' + helper.biomarker_df['rs_id'][entity_mask].astype(int).astype(str) + ' mutation in ' + helper.biomarker_df['name'][entity_mask].str.lower() + ' (' + helper.biomarker_df['gene'][entity_mask] + ')'
helper.populate_col(source = bio_entity_series, target_col = 'assessed_biomarker_entity')

# notes column 
note_series = helper.biomarker_df['notes'].copy()
uniprot_mask = helper.biomarker_df['uniprot'].notna()
note_series[uniprot_mask] = 'UPKB:' + helper.biomarker_df['uniprot'][uniprot_mask]
mutation_mask = helper.biomarker_df['mutation'].notna()
note_series[mutation_mask] = note_series[mutation_mask] + ' (' + helper.biomarker_df['mutation'][mutation_mask] + ')'
note_series + '|GRCh38'
helper.populate_col(source = note_series, target_col = 'notes')

helper.biomarker_df

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id,gene,disease,uniprot,name,system,doid,mutation,variation
0,NaN,dbSNP:rs1800562,rs1800562 mutation in hereditary haemochromato...,presence of,risk_biomarker,blood (UN:178),48579-7,hereditary cancer-predisposing syndromeDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UK37 (NM_000410.4(HFE):c.845G>A (p.Cys2...,1800562,HFE,Hereditary cancer-predisposing syndrome,Q9UK37,Hereditary haemochromatosis protein (Hereditar...,Bld/Tiss,nan,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),9
1,NaN,dbSNP:rs121908830,rs121908830 mutation in transmembrane protein ...,presence of,risk_biomarker,nan,nan,hereditary cancer-predisposing syndromeDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:C9J4H2 (NM_017849.4(TMEM127):c.475C>T (p....,121908830,TMEM127,Hereditary cancer-predisposing syndrome,C9J4H2,Transmembrane protein 127,nan,nan,NM_017849.4(TMEM127):c.475C>T (p.Gln159Ter),108
2,NaN,dbSNP:rs137854550,rs137854550 mutation in neurofibromin (NF1),presence of,risk_biomarker,amniotic fluid (UN:173),45331-6,hereditary cancer-predisposing syndromeDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UMU3 (NM_001042492.3(NF1):c.4330A>G (p....,137854550,NF1,Hereditary cancer-predisposing syndrome,Q9UMU3,Neurofibromin,Amnio fld,nan,NM_001042492.3(NF1):c.4330A>G (p.Lys1444Glu),336
3,NaN,dbSNP:rs137854552,rs137854552 mutation in neurofibromin (NF1),presence of,risk_biomarker,amniotic fluid (UN:173),45331-6,hereditary cancer-predisposing syndromeDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UMU3 (NM_001042492.3(NF1):c.5902C>T (p....,137854552,NF1,Hereditary cancer-predisposing syndrome,Q9UMU3,Neurofibromin,Amnio fld,nan,NM_001042492.3(NF1):c.5902C>T (p.Arg1968Ter),343
4,NaN,dbSNP:rs137854560,rs137854560 mutation in neurofibromin (NF1),presence of,risk_biomarker,amniotic fluid (UN:173),45331-6,hereditary cancer-predisposing syndromeDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UMU3 (NM_001042492.3(NF1):c.4084C>T (p....,137854560,NF1,Hereditary cancer-predisposing syndrome,Q9UMU3,Neurofibromin,Amnio fld,nan,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220933,NaN,dbSNP:rs<NA>,mutation in DNA repair protein RAD50(RAD50),presence of,risk_biomarker,nan,nan,hereditary cancer-predisposing syndromeDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:H7C0V2 (NM_005732.4(RAD50):c.3668T>G (p.L...,<NA>,RAD50,Hereditary cancer-predisposing syndrome,H7C0V2,DNA repair protein RAD50,nan,nan,NM_005732.4(RAD50):c.3668T>G (p.Leu1223Arg),2134861
220934,NaN,dbSNP:rs<NA>,mutation in Breast and ovarian cancer suscepti...,presence of,risk_biomarker,blood (UN:178),79208-5,familial cancer of breastDOID:nan),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9H4L3 (NM_000059.4(BRCA2):c.6916_6917ins...,<NA>,BRCA2,Familial cancer of breast,Q9H4L3,Breast and ovarian cancer susceptibility prote...,Bld/Tiss,nan,NM_000059.4(BRCA2):c.6916_6917insA (p.Ala2306fs),2412590
220935,NaN,dbSNP:rs<NA>,mutation in Breast and ovarian cancer suscepti...,presence of,risk_biomarker,blood (UN:178),79208-5,hereditary breast ovarian cancer syndromeDOID:...,gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9H4L3 (NM_000059.4(BRCA2):c.7171G>T (p.G...,<NA>,BRCA2,Hereditary breast ovarian cancer syndrome,Q9H4L3,Breast and ovarian cancer susceptibility prote...,Bld/Tiss,5683.0,NM_000059.4(BRCA2):c.7171G>T (p.Glu2391Ter),2429251
220936,NaN,dbSNP:rs<NA>,mutation in Breast and ovarian cancer suscepti...,presence of,risk_biomarker,blood (UN:178),79208-5,hereditary breast ovarian cancer syndromeDOID:...,gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9H4L3 (NM_000059.4(BRCA2):c.3189del (p.G...,<NA>,BRCA2,